In [3]:
from langchain.memory import ConversationBufferMemory

#return_message 는 챗모델을 위한 거면 return_messages =True로 변환
memory = ConversationBufferMemory(return_messages=False)
memory.save_context({"input":"Hi"},{"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='How are you?')]}

In [4]:
from langchain.memory import ConversationBufferWindowMemory

# 매개변수 k 는 몇 개의 메세지를 저장하고 있는지 first come first out
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})
    
add_message(1,1)

In [5]:
add_message(2,2)
add_message(3,3)
add_message(4,4)


In [6]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [7]:
add_message(5,5)


In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

###ConversationSummaryMemory
긴 대화를 할 때 요약을 해주므로 대화가 길 때 유용하다

In [12]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 매개변수 k 는 몇 개의 메세지를 저장하고 있는지 first come first out
memory = ConversationSummaryMemory(llm=llm)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea","wow that is so cool")
    

In [13]:
add_message("South Korea is so pretty","I wish I could go")

In [14]:
get_history()

{'history': 'Nicolas introduces himself as living in South Korea. The AI responds with enthusiasm, calling it cool. The human mentions how pretty South Korea is, and the AI expresses a wish to visit.'}

가장 중요한 부분만 저장

In [15]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 매개변수 k 는 몇 개의 메세지를 저장하고 있는지 first come first out
memory = ConversationKGMemory(llm=llm, return_messages=True)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})
    
add_message("Hi I'm Nicolas, I live in South Korea","wow that is so cool")

In [16]:
memory.load_memory_variables({"input" : "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.')]}

In [8]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature= 0.1)

template=""" 
    You are a helpful AI talking to a human.
    {chat_history}
    Human: {question}
    You:
"""

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
    memory_key="history",
    )

chain = LLMChain(
    llm=llm, 
    memory=memory, 
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Won")



> Entering new LLMChain chain...
Prompt after formatting:
 
    You are a helpful AI talking to a human.
    
    Human: My name is Won
    You:


> Finished chain.


'Hello Won! How can I assist you today?'

In [9]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
 
    You are a helpful AI talking to a human.
    Human: My name is Won
AI: Hello Won! How can I assist you today?
    Human: I live in Seoul
    You:


> Finished chain.


"That's great to hear! How can I assist you with anything related to Seoul or any other topic you have in mind?"

In [10]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
 
    You are a helpful AI talking to a human.
    Human: My name is Won
AI: Hello Won! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with anything related to Seoul or any other topic you have in mind?
    Human: what is my name?
    You:


> Finished chain.


'Your name is Won.'

In [7]:
## 우리가 history를 저장해야 한다. 메모리에 요약되고 있는 문장을 다시 prompt에 넣어야 한다.

memory.load_memory_variables({})

{'history': "System: The human introduces themselves as Won and mentions they live in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. The AI expresses their love for exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain, highlighting Seoul's dynamic energy and charm.\nHuman: what is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}

In [ ]:
### Chat Based Memory

In [11]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder

llm = ChatOpenAI(temperature= 0.1)



memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
    memory_key="history",
    return_messages = True
    )

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")
])

chain = LLMChain(
    llm=llm, 
    memory=memory, 
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Won")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Won

> Finished chain.


'Hello Won! How can I assist you today?'

In [12]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Won
AI: Hello Won! How can I assist you today?
Human: I live in Seoul

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'

In [14]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Won
AI: Hello Won! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: what is my name?

> Finished chain.


'Your name is Won. How can I assist you further, Won?'

###LCEL Based Memory

In [19]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder

llm = ChatOpenAI(temperature= 0.1)



memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
    # memory_key="chat_history",
    return_messages = True
    )

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")
])
# "_"로 무시해 버린다. load_memory에 "My name is won" 인풋이 자동으로 들어가기 때문에
def load_memory(_):
    return memory.load_memory_variables({})["history"]

#단계 : 먼저 load_memory 함수에서 요약된 history를 가져온 후 prompt의 MessagePlaceholder에 넣는다.
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):

    result = chain.invoke({
        "question" : question
    })
    memory.save_context({"input":question},{"output":result.content})
    print(result)

In [20]:
invoke_chain("my name is nico")

content='Hello Nico! How can I assist you today?'
